<a href="https://colab.research.google.com/github/TZajas/CS4287-CNN-Project/blob/main/CS4287_Assign2_20278748_20272626_20273118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS4287 CNN Project**

---

In [ ]:
# Tomasz Zajas: 20278748 | Cael O'Flaherty: 20272626 | Dawid Sobczak: 20273118
# Runs with no errors

In [ ]:
# Necessary libraries
from IPython.display import display

# For processing dataset
import numpy as np
import pandas as pd

# For plotting dataset and model
import seaborn as sb
import matplotlib.pyplot as plt

# For creating and training dataset
import tensorflow as tf

## Dataset Exploration and Visualisation

---

## Data Preprocessing and Normalization

---



## Model Definition

---



## Model Training

---



## Model Evaluation

---



## Cross Fold Validation

---

